In [1]:

from gensim.corpora import Dictionary
from gensim.models.coherencemodel import CoherenceModel
from gensim.models.ldamodel import LdaModel
from pprint import pprint

import re
import pyLDAvis.gensim
import pandas as pd

In [2]:
tweets = pd.read_csv('tweets_negative.csv')


In [3]:
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
nltk.download('punkt')
nltk.download('stopwords')
stop_words = set(stopwords.words('english'))

def remove_urls(text):
    url_pattern = re.compile(r'https?://\S+|www\.\S+')
    return url_pattern.sub('', text)

def remove_stopwords(text):
    tokens = word_tokenize(text)
    filtered_tokens = [word for word in tokens if word.lower() not in stop_words]
    return ' '.join(filtered_tokens)

def remove_mentions(text):
    mention_pattern = re.compile(r'@\w+')
    return mention_pattern.sub('', text)

def remove_special_chars(text):
    clean_text = re.sub(r'[^a-zA-Z0-9\s]', '', text)
    return clean_text


tweets['text'] = tweets['text'].str.lower()
tweets['text'] = tweets['text'].apply(remove_urls)
tweets['text'] = tweets['text'].apply(remove_stopwords)
tweets['text'] = tweets['text'].apply(remove_mentions)
tweets['text'] = tweets['text'].apply(remove_special_chars)




[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\yaori\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\yaori\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [4]:
tweets.head()

,text
0,congratulations mayorros reelection hope camp...
1,good point hs2 nt environmental win especial...
2,lib dem surge antihs2 lib dem stephen lambert ...
3,surprisentoo many snouts trough incl crayonp...
4,hs2 meadowhall fiasco syorks would metro mayo...


In [5]:
tweets = tweets.text.values.tolist()

# Turn the list of string into a list of tokens
tweets = [t.split() for t in tweets]

In [6]:
tweets[0]

['congratulations',
 'mayorros',
 'reelection',
 'hope',
 'campaign',
 'damaging',
 'untruthful',
 'hs2',
 'new',
 'route',
 'meets',
 'equal',
 'success']

In [7]:
id2word = Dictionary(tweets)
# Term Document Frequency
corpus = [id2word.doc2bow(text) for text in tweets]
print(corpus[:1])

[[(0, 1), (1, 1), (2, 1), (3, 1), (4, 1), (5, 1), (6, 1), (7, 1), (8, 1), (9, 1), (10, 1), (11, 1), (12, 1)]]


In [17]:
# Build LDA model
lda_model = LdaModel(corpus=corpus,
                     id2word=id2word,
                     num_topics=5,
                     random_state=0,
                     chunksize=100,
                     alpha='auto',
                     per_word_topics=True)

pprint(lda_model.print_topics())
doc_lda = lda_model[corpus]

[(0,
  '0.073*"hs2" + 0.011*"rail" + 0.010*"government" + 0.010*"people" + '
  '0.009*"going" + 0.009*"protesters" + 0.009*"like" + 0.008*"still" + '
  '0.007*"news" + 0.007*"case"'),
 (1,
  '0.030*"hs2" + 0.030*"nn" + 0.018*"report" + 0.014*"cost" + 0.013*"scrap" + '
  '0.012*"construction" + 0.011*"costs" + 0.011*"s" + 0.009*"hs2ltd" + '
  '0.007*"lockdown"'),
 (2,
  '0.031*"hs2" + 0.023*"n" + 0.016*"mps" + 0.014*"stop" + 0.012*"post" + '
  '0.010*"another" + 0.010*"please" + 0.008*"anti" + 0.007*"bosses" + '
  '0.006*"trees"'),
 (3,
  '0.116*"hs2" + 0.014*"borisjohnson" + 0.013*"amp" + 0.013*"project" + '
  '0.011*"money" + 0.011*"s" + 0.010*"nt" + 0.009*"need" + 0.009*"work" + '
  '0.009*"time"'),
 (4,
  '0.039*"hs2" + 0.034*"stophs2" + 0.016*"workers" + 0.012*"petition" + '
  '0.009*"via" + 0.008*"sites" + 0.008*"take" + 0.006*"destruction" + '
  '0.006*"ever" + 0.006*"scrapped"')]


In [18]:
# Compute Coherence Score
coherence_model_lda = CoherenceModel(model=lda_model, texts=tweets, dictionary=id2word, coherence='c_v')
coherence_lda = coherence_model_lda.get_coherence()
print('\nCoherence Score: ', coherence_lda)


Coherence Score:  0.3110218160892245


In [19]:

#Creating Topic Distance Visualization
pyLDAvis.enable_notebook()
pyLDAvis.gensim.prepare(lda_model, corpus, id2word)

PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
3     -0.202157  0.138684       1        1  35.271678
0     -0.104201 -0.204869       2        1  24.162975
2      0.099846  0.043982       3        1  13.886149
4      0.099734  0.010479       4        1  13.555716
1      0.106778  0.011725       5        1  13.123482, topic_info=              Term        Freq        Total Category  logprob  loglift
1197       stophs2  444.000000   444.000000  Default  30.0000  30.0000
300             nn  326.000000   326.000000  Default  29.0000  29.0000
1466             n  354.000000   354.000000  Default  28.0000  28.0000
1658  borisjohnson  413.000000   413.000000  Default  27.0000  27.0000
414         report  195.000000   195.000000  Default  26.0000  26.0000
...            ...         ...          ...      ...      ...      ...
5              hs2  327.587477  6014.499585   Topic5  -3.5104  -0.8794
9575     nhsnoths2   62.337364   381.717788   Topic5  -5.1696   0.2186
416            see   46.034758   132.451529   Topic5  -5.4728   0.9739
2702           lot   38.443140    68.779945   Topic5  -5.6530   1.4490
9643   coronavirus   37.999283   213.081755   Topic5  -5.6646   0.3067

[252 rows x 6 columns], token_table=      Topic      Freq     Term
term                          
271       3  0.528341        2
271       4  0.463909        2
477       1  0.993608       20
1494      5  0.982823     2017
1052      4  0.973303  account
...     ...       ...      ...
69        4  0.003863    would
446       4  0.982081     year
580       1  0.989021      yes
216       3  0.965647      yet
216       4  0.020546      yet

[327 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[4, 1, 3, 5, 2])